# Import Library

In [3]:
!pip install Sastrawi
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import numpy as np
import pickle

In [4]:
dataset= pd.read_csv("https://drive.usercontent.google.com/download?id=1tZ3_aGG_F7Okl_35y3Fmm9K_mpQiMpWb&export=download&authuser=0&confirm=t&uuid=50a5072f-f529-4fdb-85b5-51a129083916&at=APcmpoz7ZE6qWvmrkR_uG1eE-K8N:1744013439361")
dataset.head(2)

,place_name,place_description,category,city,price,rating,description_location,place_img,gallery_photo_img1,gallery_photo_img2,gallery_photo_img3,place_map,user_ratings_total,longitude,latitude
0,Air Terjun Takapala,Air Terjun Takapala di Sulawesi Selatan mempes...,"tourist_attraction, point_of_interest, establi...",Sulawesi Selatan 92174,Rp5.000,4.5,"PVF5+HW, Bonto Lerung, Kec. Tinggimoncong, Kab...",https://maps.googleapis.com/maps/api/place/pho...,NaN,NaN,NaN,https://www.google.com/maps/place/?q=place_id:...,1952.0,1.198.569.436,-5.274.659.199.999.990
1,Pantai Biluhu Timur,Pantai Biluhu Timur di Gorontalo menawarkan pe...,"tourist_attraction, point_of_interest, establi...",Gorontalo,Rp5.000,4.5,"FXRF+59J, Biluhu Tim., Kec. Batudaa Pantai, Ka...",https://maps.googleapis.com/maps/api/place/pho...,NaN,NaN,NaN,https://www.google.com/maps/place/?q=place_id:...,159.0,1.229.734.002,4.904.571


In [ ]:
df = dataset[['place_name','place_description','category','city','rating','description_location','longitude','latitude','user_ratings_total']]
# Handle missing values
text_columns = ['place_description', 'description_location']
for col in text_columns:
    df[col] = df[col].fillna('')

categorical_columns = ['category', 'city']
for col in categorical_columns:
    df[col] = df[col].fillna(df[col].mode()[0])

numeric_columns = ['rating']
for col in numeric_columns:
    df[col] = df[col].fillna(df[col].median())

# Combine text features
df['combined_text'] = df['place_description'] + " " + df['description_location']
df['combined_text'] = df['combined_text'].apply(lambda x: stemmer.stem(x))
df['combined_text'] = df['combined_text'].apply(lambda x: x.lower())
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Define Indonesian stopwords
indonesian_stopwords = ['yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'untuk', 'pada', 'dengan',
                        'adalah', 'atau', 'jika', 'saya', 'kita', 'akan', 'tidak', 'tersebut']

# Create TF-IDF features
vectorizer = TfidfVectorizer(stop_words=indonesian_stopwords, max_features=1000)
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

# Encode categorical features
le_category = LabelEncoder()
df['category_encoded'] = le_category.fit_transform(df['category'])

le_city = LabelEncoder()
df['city_encoded'] = le_city.fit_transform(df['city'])

# Scale numerical features
scaler = StandardScaler()
df[['rating_scaled']] = scaler.fit_transform(df[['rating']])

# Combine all features
other_features = df[['category_encoded', 'city_encoded', 'rating_scaled']]
other_features_sparse = sp.csr_matrix(other_features.values)
final_feature_matrix = sp.hstack([tfidf_matrix, other_features_sparse])


In [6]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(final_feature_matrix, dense_output=False)

# Recommendation function
def get_recommendations(place_index, similarity_matrix, df, top_n=5):
    similarity_scores = list(enumerate(similarity_matrix[place_index].toarray().flatten()))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    recommended_indices = [i[0] for i in similarity_scores]
    return recommended_indices

In [10]:
# Get recommendations
recommended_indices = get_recommendations(0, similarity_matrix, dataset, top_n=5)
recommended_places = dataset.iloc[recommended_indices]
print("Recommended indices:", recommended_indices)
print(recommended_places[['place_name', 'category', 'city', 'rating',
                          'place_img', 'gallery_photo_img1', 'gallery_photo_img2', 'gallery_photo_img3']])

Recommended indices: [102, 512, 22, 656, 262]
                       place_name  \
102           Parangloe Waterfall   
512         Wisata Alam Wai Tiddo   
22         Pantai Barugaya Punaga   
656  Taman Wisata Holyland Malino   
262                   Buttu Macca   

                                              category  \
102  tourist_attraction, point_of_interest, establi...   
512  tourist_attraction, point_of_interest, establi...   
22   tourist_attraction, point_of_interest, establi...   
656  tourist_attraction, point_of_interest, establi...   
262  tourist_attraction, point_of_interest, establi...   

                       city  rating  \
102  Sulawesi Selatan 92173     4.5   
512  Sulawesi Selatan 91991     4.6   
22   Sulawesi Selatan 92261     4.4   
656  Sulawesi Selatan 92173     4.4   
262  Sulawesi Selatan 91752     4.4   

                                             place_img gallery_photo_img1  \
102  https://maps.googleapis.com/maps/api/place/pho...                

In [8]:
# Simpan model ke file pickle
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump([similarity_matrix, vectorizer, le_category, le_city, scaler, df], f)

**Variabel yang Disimpan**
  * similarity_matrix: Matriks kesamaan kosinus.
  * vectorizer: Objek TF-IDF Vectorizer.
  * le_category: Label Encoder untuk kategori.
  * le_city: Label Encoder untuk kota.
  * scaler: Standard Scaler untuk rating.
  * df: DataFrame yang berisi data tempat (diganti dari dataset untuk konsistensi).
---
**Cara Memuat Model:**


```
import pickle
with open('recommendation_model.pkl', 'rb') as f:
    similarity_matrix, vectorizer, le_category, le_city, scaler, df = pickle.load(f)

```




